# Setup

In [0]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
import pandas as pd
import numpy as np
import logging ### to monitor the code
from bs4 import BeautifulSoup
import pickle
import math
import random
import sys
import csv
import unicodedata

import nltk
from nltk.collocations import *

import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import xml.etree.cElementTree as ET
from urllib.request import urlopen

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

!pip install sddk
import sddk

In [0]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [0]:
### authorize google sheets
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
# establish connection with particular sheet by its url:

PIA_data = gc.open_by_url("https://docs.google.com/spreadsheets/d/1KxOx7Be9fj3lDcEPgQhQ-Iqcn9p367-MMD6RMXe8rks/edit?usp=sharing")
PIA_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1e94wyelg6dftQ4zxbq1xvwxWAI-BhcYXtclDW-YTnrw/edit?usp=sharing")

In [0]:
PIA_data.worksheets()

[<Worksheet 'Sheet1' id:0>,
 <Worksheet 'Aristotle_λύπη' id:1718022011>,
 <Worksheet 'Hippocrates_λύπη' id:309444251>,
 <Worksheet 'Hippocrates_ἀλγηδών' id:844140843>,
 <Worksheet 'Hippocrates_πόνος' id:1101646330>,
 <Worksheet 'Hippocrates_ὀδύνη' id:287961093>,
 <Worksheet 'Aristotle_ἀλγηδών' id:1146799700>,
 <Worksheet 'Aristotle_πόνος' id:1073775859>,
 <Worksheet 'Aristotle_ὀδύνη' id:438127122>,
 <Worksheet 'keyterms_overview' id:1994560877>,
 <Worksheet 'c_aristotelicum_OVERVIEW_NEW' id:984710941>,
 <Worksheet 'c_hippocraticum_OVERVIEW_NEW' id:1964402831>]

# Load the main dataset of ancient Greek texts

In [0]:
AGT = sddk.read_file("SDAM_data/AGT/AGT_20200506.json", "df", conf)

In [0]:
AGT.head(5)

,filename,author,title,string,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,wordcount,lemmata,lemmatized_sentences,lemmata_wordcount,num_of_sents
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica; Argonautica,"ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...",tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},2.5,pagan,38822,"[σέο, φοῖβε, παλαιγενής, κλέος, φάος, μιμνήσκω...","[[σέο, φοῖβε, παλαιγενής, κλέος, φάος, μιμνήσκ...",24348,3229
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War; Historiae in two volumes,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν ...,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,150160,"[θουκυδίδης, ἀθηναῖος, συγγράφω, πόλεμος, πελο...","[[θουκυδίδης, ἀθηναῖος, συγγράφω, πόλεμος, πελ...",68349,6091
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers; Lives of Eminen...,ΒΙΩΝ ΚΑΙ ΓΝΩΜΩΝ ΤΩΝ ΕΝ ΦΙΛΟΣΟΦΙΑΙ ΕΥΔΟΚΙΜΗΣΑΝΤ...,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,pagan,111053,"[βιων, και, γνωμων, των, εν, φιλοσοφιαι, ευδοκ...","[[βιων, και, γνωμων, των, εν, φιλοσοφιαι, ευδο...",55950,10065
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls ; Idylls,Θύρσις ἢ ᾠδή Θύρσις ̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ...,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,pagan,19514,"[θύρσις, ἀοιδή, θύρσις, αδύ, ψιθύρισμα, πίτυς,...","[[θύρσις, ἀοιδή, θύρσις, αδύ, ψιθύρισμα, πίτυς...",12522,1726
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams ; Idylls,α παλ ϝι.336 τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυ...,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,pagan,1807,"[παλ, ϝι, ῥοδῆ, δροσόεντα, κατάπυκνος, ἐκεῖνος...","[[παλ, ϝι], [ῥοδῆ, δροσόεντα, κατάπυκνος, ἐκεῖ...",1142,158


# Data Extraction and Overview



In [0]:
### how many documents we have
len(AGT)

1182

In [0]:
### let's identify our main author of interest

c_hippocraticum = AGT[AGT["author_id"].str.startswith("tlg0627")]
len(c_hippocraticum) ### old value: 53

52

In [0]:
c_hippocraticum["wordcount"].sum() # old value: 443514

334392

In [0]:
set_with_dataframe(PIA_overview.add_worksheet("c_hippocraticum_OVERVIEW", rows=1, cols=1), c_hippocraticum.drop(["lemmata", "string", "lemmatized_sentences",], axis=1))

In [0]:
c_aristotelicum = AGT[AGT["author_id"].str.startswith("tlg0086")]
len(c_aristotelicum) ### originally we had 27

35

In [0]:
c_aristotelicum["wordcount"].sum()

859860

However in the case of Aristotle, we are interested only in a subselection of works associated with his name




In [0]:
# read back the manually coded data
c_aristotelicum_coded = get_as_dataframe(PIA_data.worksheet("c_aristotelicum_OVERVIEW_NEW"))
c_aristotelicum_coded.head()

,filename,include?,author,title,author_id,doc_id,wordcount,lemmata_wordcount,num_of_sents
0,tlg0086.tlg001.1st1K-grc2.xml,y,Aristotle,Aristotelis Analytica Priora et Posteriora; Ar...,tlg0086,tlg0086.tlg001,59772,12287,3384
1,tlg0086.tlg002.1st1K-grc2.xml,y,Aristotle,De anima; Aritoteles De anima,tlg0086,tlg0086.tlg002,20988,5579,1250
2,tlg0086.tlg003.perseus-grc1.xml,NaN,Aristotle,Athenian Constitution; Machine readable text; ...,tlg0086,tlg0086.tlg003,16536,4243,817
3,tlg0086.tlg005.1st1K-grc1.xml,y,Aristotle,De caelo; Aristoteles De coelo et De generatio...,tlg0086,tlg0086.tlg005,31395,8370,1856
4,tlg0086.tlg006.1st1K-grc1.xml,y,Aristotle,Categoriae; Aristotelis Opera,tlg0086,tlg0086.tlg006,10317,2865,646


In [0]:
### we can check whether the new and old version have the same name
print(c_aristotelicum_coded["doc_id"].tolist())
print(c_aristotelicum["doc_id"].tolist())

['tlg0086.tlg001', 'tlg0086.tlg002', 'tlg0086.tlg003', 'tlg0086.tlg005', 'tlg0086.tlg006', 'tlg0086.tlg008', 'tlg0086.tlg009', 'tlg0086.tlg010', 'tlg0086.tlg013', 'tlg0086.tlg014', 'tlg0086.tlg015', 'tlg0086.tlg016', 'tlg0086.tlg017', 'tlg0086.tlg018', 'tlg0086.tlg020', 'tlg0086.tlg021', 'tlg0086.tlg022', 'tlg0086.tlg024', 'tlg0086.tlg025', 'tlg0086.tlg026', 'tlg0086.tlg029', 'tlg0086.tlg030', 'tlg0086.tlg031', 'tlg0086.tlg034', 'tlg0086.tlg035', 'tlg0086.tlg037', 'tlg0086.tlg038', 'tlg0086.tlg040', 'tlg0086.tlg041', 'tlg0086.tlg042', 'tlg0086.tlg043', 'tlg0086.tlg044', 'tlg0086.tlg045', 'tlg0086.tlg052', 'tlg0086.tlg054']
['tlg0086.tlg001', 'tlg0086.tlg002', 'tlg0086.tlg003', 'tlg0086.tlg005', 'tlg0086.tlg006', 'tlg0086.tlg008', 'tlg0086.tlg009', 'tlg0086.tlg010', 'tlg0086.tlg013', 'tlg0086.tlg014', 'tlg0086.tlg015', 'tlg0086.tlg016', 'tlg0086.tlg017', 'tlg0086.tlg018', 'tlg0086.tlg020', 'tlg0086.tlg021', 'tlg0086.tlg022', 'tlg0086.tlg024', 'tlg0086.tlg025', 'tlg0086.tlg026', 'tlg0086

In [0]:
# 1) add the "include?" column from the old overview to our new dataset 
c_aristotelicum["include?"] = c_aristotelicum_coded["include?"].tolist()
# 2) use the "include?" column for filtering the new dataset
# 3) drop "include?" column 
c_aristotelicum = c_aristotelicum[c_aristotelicum["include?"]=="y"].drop("include?", axis=1)
c_aristotelicum.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,filename,author,title,string,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,wordcount,lemmata,lemmatized_sentences,lemmata_wordcount,num_of_sents
678,tlg0086.tlg001.1st1K-grc2.xml,Aristotle,Aristotelis Analytica Priora et Posteriora; Ar...,ΑΝΑΛΥΤΙΚΩΝ ΠΡΟΤΕΡΩΝ Α. Πρῶτον εἰπεῖν περὶ τί κ...,tlg0086,tlg0086.tlg001,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,59772,"[αναλυτικων, προτερων, πρῶτον, εἶπον, σκέψις, ...","[[αναλυτικων, προτερων], [πρῶτον, εἶπον, σκέψι...",21150,3645
679,tlg0086.tlg002.1st1K-grc2.xml,Aristotle,De anima; Aritoteles De anima,ΠEP1 ΨΥΧΗΣ Α Τῶν καλῶν καὶ τιμίων τὴν εἴδησιν ...,tlg0086,tlg0086.tlg002,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,20988,"[ψυχης, κᾶλον, τίμιος, εἴδομαι, μαλλός, ἕτερος...","[[ψυχης, κᾶλον, τίμιος, εἴδομαι, μαλλός, ἕτερο...",8893,1306
681,tlg0086.tlg005.1st1K-grc1.xml,Aristotle,De caelo; Aristoteles De coelo et De generatio...,ΠEPI ΟΥΡΑΝΟΥ A. Ἡ περὶ φύσεως ἐπιστήμη σχεδὸν ...,tlg0086,tlg0086.tlg005,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,31395,"[ουρανου, φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶ...","[[ουρανου], [φύσις, ἐπιστήμη, πλεῖστος, φαίνω,...",12915,1986
682,tlg0086.tlg006.1st1K-grc1.xml,Aristotle,Categoriae; Aristotelis Opera,ΚΑΤΗΓΟΡΙΑΙ. ὉΜΩΝΥΜΑ λέγεται ὧν ὄνομα μόνον κοι...,tlg0086,tlg0086.tlg006,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,10317,"[κατηγοριαι, ὁμωνυμα, λέγω, ὀνομάζω, μόνος, κο...","[[κατηγοριαι], [ὁμωνυμα, λέγω, ὀνομάζω, μόνος,...",4372,659
683,tlg0086.tlg008.1st1K-grc1.xml,Aristotle,De divinatione per somnum; Aristotelis Opera,ΠΕΡΙ ΕΝΥΠΝΙΩΝ ΚΑΙ ΤΗΣ ΚΑΘ’ ΥΠΝΟΝ ΜΑΝΤΙΚΗΣ. Περ...,tlg0086,tlg0086.tlg008,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,1207,"[περι, ενυπνιων, και, της, καθ, υπνον, μαντικη...","[[περι, ενυπνιων, και, της, καθ, υπνον, μαντικ...",533,69


In [0]:
len(c_aristotelicum)

27

In [0]:
c_aristotelicum["wordcount"].sum()

803085

In [0]:
set_with_dataframe(PIA_overview.add_worksheet("c_aristotelicum_OVERVIEW_NEW", rows=1, cols=1), c_aristotelicum.drop(["lemmata", "string", "lemmatized_sentences"], axis=1))

# Export the subcorpora into sciencedata

We will export the files into Vojtěch's directory on sciencedata.dk, which will be later on made public.

In [0]:
conf = sddk.configure()

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
endpoint variable has been configured to: https://sciencedata.dk/files/


In [0]:
###publicfolder = "31b393e2afe1ee96ce81869c7efe18cb"
### sddk.write_file("https://sciencedata.dk/public" + publicfolder + "/c_hippocraticum.json", c_hippocraticum)


Type shared folder name or press Enter to skip: 
sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
endpoint variable has been configured to: https://sciencedata.dk/files/
The path is not valid. Try different path and filename: public_data/PIA
A file with the same name ("PIA") already exists in this location.
Press Enter to overwrite it or choose different path and filename: public_data/PIA/c_hippocraticum.json
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/public_data/PIA/c_hippocraticum.json"


In [0]:
sddk.write_file("public_data/PIA/c_hippocraticum.json", c_hippocraticum, conf)

A file with the same name ("c_hippocraticum.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/public_data/PIA/c_hippocraticum.json"


In [0]:
sddk.write_file("public_data/PIA/c_aristotelicum.json", c_aristotelicum, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/public_data/PIA/c_aristotelicum.json"
